In [1]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from FinMind.data import DataLoader
import matplotlib.pyplot as plt
from neuralprophet import NeuralProphet
from statsmodels.tsa.stattools import pacf
import numpy as np

api = DataLoader()

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
def best_pacf_lag(data, max_lags=30):
    pacf_values = pacf(data, nlags=max_lags, method="ywadjusted")
    conf_int = 1.96 / np.sqrt(len(data))  # 95% confidence threshold

    # Find first lag where PACF falls within confidence bounds
    for lag in range(1, len(pacf_values)):
        if abs(pacf_values[lag]) < conf_int:
            return lag - 1

    return max_lags

In [4]:
today = datetime.today()
start_date = (today - relativedelta(years=5)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

ticker='2609'
df = api.taiwan_stock_daily(
    stock_id=ticker, start_date=start_date, end_date=end_date
)
df = df[["date", "close"]]
df.columns = ["ds", "y"]
df["ds"] = pd.to_datetime(df["ds"])

print('lags', best_pacf_lag(df["y"]))
# 訓練模型
m = NeuralProphet(
    n_changepoints=0,
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    n_lags=best_pacf_lag(df["y"])
)
m.fit(df, epochs=100, batch_size=32, early_stopping=True)
forecast = m.predict(df)
m.plot_components(forecast)

2025-12-07 18:03:25.705 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 2609


lags 1


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/neuralprophet/df_utils.py:464: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


WARNING - (py.warnings._showwarnmsg) - /Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 96.299% of the data.
INFO - (NP.df_utils._in

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (41) is too small than the required number                     for the learning rate finder (228). The results might not be optimal.
Finding best initial lr: 100%|██████████| 228/228 [00:08<00:00, 26.85it/s]


...


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 131.07it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
ERROR - (NP.plotly.plot_components) - plotly-resampler is not installed. Please install it to use the resampler.


WARNING - (py.warnings._showwarnmsg) - /Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/neuralprophet/plot_forecast_plotly.py:410: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - /Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/neuralprophet/plot_forecast_plotly.py:559: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result




ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'Trend',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2020, 12, 8, 0, 0),
                          datetime.datetime(2020, 12, 9, 0, 0),
                          datetime.datetime(2020, 12, 10, 0, 0), ...,
                          datetime.datetime(2025, 12, 3, 0, 0),
                          datetime.datetime(2025, 12, 4, 0, 0),
                          datetime.datetime(2025, 12, 5, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('VNRwwoTucMK2CHHC6CJxwn5xccKwi3' ... 'XCB3rVwiCH1cI4lNXCUaHVwmqu1cI='),
                    'dtype': 'f4'},
              'yaxis': 'y'},
             {'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'yearly seasonality',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2020, 12, 8, 0, 0),
                          datetime.datetime(2020, 12, 9, 0, 0),
                          datetime.datetime(2020, 12, 10, 0, 0), ...,
                          datetime.datetime(2025, 12, 3, 0, 0),
                          datetime.datetime(2025, 12, 4, 0, 0),
                          datetime.datetime(2025, 12, 5, 0, 0)], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': ('g+1OwAACOcDPmCPA9NEOwFPFqr8Meo' ... 'jALR25wMQLrsAK0qLAlYCXwGUdjMA='),
                    'dtype': 'f4'},
              'yaxis': 'y2'},
             {'marker': {'color': '#2d92ff'},
              'name': 'Auto-Regression',
              'opacity': 1.0,
              'showlegend': False,
              'type': 'bar',
              'x': array([datetime.datetime(2020, 12, 8, 0, 0),
                          datetime.datetime(2020, 12, 9, 0, 0),
                          datetime.datetime(2020, 12, 10, 0, 0), ...,
                          datetime.datetime(2025, 12, 3, 0, 0),
                          datetime.datetime(2025, 12, 4, 0, 0),
                          datetime.datetime(2025, 12, 5, 0, 0)], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': ('JueYQk+nmkI1uJpCzwCaQuwbmEJbKp' ... 'VD/BYlQxKTJEMQ2yNDb24jQwAAAAA='),
                    'dtype': 'f4'},
              'yaxis': 'y3'}],
    'layout': {'autosize': True,
               'barmode': 'overlay',
               'font': {'size': 10},
               'height': 630,
               'hovermode': 'x unified',
               'legend': {'traceorder': 'reversed', 'y': 0.1},
               'margin': {'b': 0, 'l': 0, 'pad': 0, 'r': 10, 't': 10},
               'template': '...',
               'title': {'font': {'size': 12}},
               'width': 700,
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'linewidth': 1.5,
                         'mirror': True,
                         'range': [2020-09-08 00:00:00, 2026-03-06 00:00:00],
                         'showline': True,
                         'title': {'text': 'ds'},
                         'type': 'date'},
               'xaxis2': {'anchor': 'y2',
                          'domain': [0.0, 1.0],
                          'linewidth': 1.5,
                          'mirror': True,
                          'range': [2020-09-08 00:00:00, 2026-03-06 00:00:00],
                          'showline': True,
                          'title': {'text': 'ds'},
                          'type': 'date'},
               'xaxis3': {'anchor': 'y3',
                          'domain': [0.0, 1.0],
                          'linewidth': 1.5,
                          'mirror': True,
                          'range': [2020-09-08 00:00:00, 2026-03-06 00:00:00],
                          'showline': True,
                          'title': {'text': 'ds'},
                          'type': 'date'},
               'yaxis': 